In [66]:
import pandas as pd
import geopandas as gpd
import requests
import json 
import random
import os
import sys
pd.options.display.max_columns = 50 #number of columns to display

In [67]:
gisJsonPath = os.path.join('Resources', 'clt_arcgis.json')

df = gpd.read_file(gisJsonPath)
df = df.set_crs("EPSG:4326")
df = df.set_crs(epsg=4326)

#df["Centroid"] = df["geometry"].centroid
#df["AREA"] = df["geometry"].area
df.head(5)

,OBJECTID,Petition,Petitioner,FromZone,ToZone,Type,SPA,Overlay,INNOV,Acres,PetAcres,Water,Sewer,CityLimit,ZoningMap,ODZ,CCW,ServiceArea,LuseStaff,UrbanDesign,RezoneStaff,Consistent,ProtestPet,Received,Approved,Decision,Hyperlink,Corrective,Propluse,AdminAppr,AdminApprD,AdminApprP,AdminApprC,SFLOTS,THUNITS,CONDOS,APT,TOTUNITS,OFFICE,RETAIL,HOTELRMS,PARKING,OPENSP,ACRESITE,SHAPE.STArea(),SHAPE.STLength(),geometry
0,565707,2014-049,"SBBH, LLC",CC,MUDD-O,CD,no,none,NO,1.53,1.53,CMUD,CMUD,YES,135,None,Mixed Use Activity Center,South,Main_Kent,Dixon_Bridget,Sanders_Sonja,None,NO,1395619200000,None,Wd,http://charlottenc.gov/planning/Rezoning/Rezon...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5.781933e+04,1024.520620,"POLYGON ((-80.82863 35.15350, -80.82877 35.153..."
1,565711,2007-054,Westport Homes of N.C.,BP(CD),R-8MF(CD),CD,no,None,None,30.90,30.90,CMUD,CMUD,YES,"127,133",163,Wedge,Southwest,Main_Kent,Vincent_Kourtnie,Fortune_Solomon,WD,None,1169424000000,None,Wd,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.448928e+06,6298.689515,"POLYGON ((-80.90417 35.16609, -80.90418 35.165..."
2,565712,1988-031,None,"R-9, R-15, 1-1(CD)",BP,None,no,None,None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,579744000000,App,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2.521052e+06,9035.692548,"POLYGON ((-80.91762 35.19226, -80.91687 35.192..."
3,565713,1988-069,None,I-2,I-1,None,no,None,None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,593049600000,App,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,3.411677e+05,3614.356960,"POLYGON ((-80.91128 35.22900, -80.91135 35.228..."
4,565715,1988-040,None,I-3,UMUD,None,no,None,None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,585187200000,App,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,4.606384e+05,2890.854571,"POLYGON ((-80.85069 35.22360, -80.85084 35.223..."


In [22]:
gisJsonFeaturesList = []

with open(gisJsonPath) as f:
    response_json = json.load(f)
    
    for feature in response_json['features']:
        feature["attributes"]["SHAPE_STArea"] = feature["attributes"]["SHAPE.STArea()"]
        del feature["attributes"]["SHAPE.STArea()"]
        
        feature["attributes"]["SHAPE_STLength"] = feature["attributes"]["SHAPE.STLength()"]
        del feature["attributes"]["SHAPE.STLength()"]
        
        gisJsonFeaturesList.append(feature["attributes"])

    

In [25]:
gisJsonDf = pd.DataFrame(gisJsonFeaturesList) 
gisJsonDf.head()
#gisJsonDf.describe

,OBJECTID,Petition,Petitioner,FromZone,ToZone,Type,SPA,Overlay,INNOV,Acres,PetAcres,Water,Sewer,CityLimit,ZoningMap,ODZ,CCW,ServiceArea,LuseStaff,UrbanDesign,RezoneStaff,Consistent,ProtestPet,Received,Approved,Decision,Hyperlink,Corrective,Propluse,AdminAppr,AdminApprD,AdminApprP,AdminApprC,SFLOTS,THUNITS,CONDOS,APT,TOTUNITS,OFFICE,RETAIL,HOTELRMS,PARKING,OPENSP,ACRESITE,SHAPE_STArea,SHAPE_STLength
0,565707,2014-049,"SBBH, LLC",CC,MUDD-O,CD,no,none,NO,1.5300,1.53,CMUD,CMUD,YES,135,None,Mixed Use Activity Center,South,Main_Kent,Dixon_Bridget,Sanders_Sonja,None,NO,1.395619e+12,NaN,Wd,http://charlottenc.gov/planning/Rezoning/Rezon...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5.781933e+04,1024.520620
1,565711,2007-054,Westport Homes of N.C.,BP(CD),R-8MF(CD),CD,no,None,None,30.9000,30.90,CMUD,CMUD,YES,"127,133",163,Wedge,Southwest,Main_Kent,Vincent_Kourtnie,Fortune_Solomon,WD,None,1.169424e+12,NaN,Wd,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.448928e+06,6298.689515
2,565716,2006-127,"Westlake Properties, LLC",B-1SCD,CC,CD,no,CR/LW-PA,None,22.0500,NaN,CMUD,CMUD,YES,"62,67",404,None,Northwest,Mccullough_Melony,Howard_John,Keplinger_Tammie,WD,None,1.151280e+12,NaN,Wd,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.362496e+06,4854.939674
3,565717,2005-074,William G. Lisk,B-2,UR-3(CD),CD,no,None,None,2.5809,1.98,CMUD,CMUD,YES,88,184,None,Central,Neely_Cheryl,Howard_John,Drake_Tom,WD,,1.109290e+12,NaN,Wd,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.124242e+05,1756.468990
4,565726,2004-082,Carolyn Hudson et al,R-22MF,R-8,NC,no,None,None,1.2650,0.88,CMUD,CMUD,YES,111,102,None,Central,Neely_Cheryl,,Waldron_Charlotte,WD,Yes,1.083024e+12,NaN,Wd,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5.510387e+04,994.235592
